# `pyscal_rdf`

Ontology based atomic structure creation, manipulation, querying.

Imports

In [1]:
from pyscal.crystal_structures import Structure
from pyscal_rdf.graph import StructureGraph

## Creation of structures

Creation of structures is carried out through pyscal. We will create three structures for the demonstration.

First, an L12 $Ni_3 Al$ structure

In [2]:
struct = Structure()

In [3]:
struct_l12 = struct.lattice.l12(element=['Al', 'Ni'], 
                         lattice_constant=3.57)

['Al', 'Ni']


Now a BCC Iron structure

In [4]:
struct_Fe = struct.element.Fe()

Fe


And finally a Si diamond structure

In [5]:
struct_Si = struct.element.Si()

Si


## Adding structures to a graph

First we create a graph

In [6]:
g = StructureGraph()

In [7]:
g.graph.parse("pyscal_rdf/data/cmso.owl", format='xml')

<Graph identifier=N304404e3864e4aef8d79731c1ae06af1 (<class 'rdflib.graph.Graph'>)>

Then add all three structures to it

In [8]:
g.add_structure_to_graph(struct_l12)
g.add_structure_to_graph(struct_Fe)
g.add_structure_to_graph(struct_Si)

In [9]:
g.graph.serialize("map.ttl", format='turtle')

<Graph identifier=N304404e3864e4aef8d79731c1ae06af1 (<class 'rdflib.graph.Graph'>)>

We can save the graph and reload it as needed

## Querying the graph

An example question would be, **what are the space group of all structures with 4 atoms?**


The corresponding SPARQL query looks like this:

In [8]:
query = """
PREFIX cmso: <https://purls.helmholtz-metadaten.de/cmso/>
SELECT DISTINCT ?symbol
WHERE {
    ?sample cmso:hasNumberOfAtoms ?number .
    ?sample cmso:hasMaterial ?material .
    ?material cmso:hasStructure ?structure .
    ?structure cmso:hasSpaceGroup ?spacegroup .
    ?spacegroup cmso:hasSpaceGroupSymbol ?symbol .
FILTER (?number="4"^^xsd:integer)
}"""

In [9]:
res = g.graph.query(query)

And print the results

In [10]:
for r in res:
    print(r[0])

Pm-3m


There are also pre-built queries. For example, **What are all the samples with Bravais lattice bcc?**

In [14]:
from pyscal_rdf.queries import Query

In [15]:
q = Query()

In [18]:
samples = q.sparql.sample_by_latticesystem(g, 'bcc')

We can check how many samples we found

In [19]:
len(samples)

1

As expected, there is only one sample

We can write this sample to a file, for example, a LAMMPS data format, to use it for further simulations

In [20]:
g.to_file(samples[0], 'bcc.data', format="lammps-data")

In [24]:
! more bcc.data

bcc.data (written by ASE) 

2 	 atoms 
1  atom types
0.0      2.8700000000000001  xlo xhi
0.0      2.8700000000000001  ylo yhi
0.0      2.8700000000000001  zlo zhi


Atoms 

     1   1                       0                       0                      
 0
     2   1      1.4350000000000001      1.4350000000000001      1.43500000000000
01


We can also export as an ASE object

In [25]:
aseobj = g.to_file(samples[0], format="ase")

In [26]:
aseobj

Atoms(symbols='Fe2', pbc=True, cell=[2.87, 2.87, 2.87])